In [ ]:
#Calculate text features for captions

In [ ]:
import pandas as pd
from random import shuffle
import numpy as np
from collections import Counter
import imp
from operator import add

import nltk
from nltk.util import ngrams
from nltk.tag.stanford import StanfordNERTagger
from sklearn.feature_extraction.text import CountVectorizer

import readability

In [ ]:
#Change these variables to work with your setup
home = 'data2016/'
TEXT_FILE = home+"text_features/allText.csv" #all caption text (throughout, you might need to change the names of the different columns to match your file)
STANFORD_NER_MODEL = '/Users/laura/software/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz'
STANFORD_NER_JAR = '/Users/laura/software/stanford-ner/stanford-ner.jar'

In [ ]:
textFeatures = pd.read_csv(TEXT_FILE)
textFeatures = textFeatures.drop('Unnamed: 0',axis=1)

In [ ]:
st = StanfordNERTagger(STANFORD_NER_MODEL,
               STANFORD_NER_JAR) 

In [ ]:
#Tokenize all comment file text
textFeatures['commentfile1_tokenized'] = [nltk.word_tokenize(i) for i in textFeatures['commentfile1'].fillna('')]
textFeatures['commentfile2_tokenized'] = [nltk.word_tokenize(i) for i in textFeatures['commentfile2'].fillna('')]
textFeatures['commentfile3_tokenized'] = [nltk.word_tokenize(i) for i in textFeatures['commentfile3'].fillna('')]
textFeatures['commentfile4_tokenized'] = [nltk.word_tokenize(i) for i in textFeatures['commentfile4'].fillna('')]
textFeatures['commentfile5_tokenized'] = [nltk.word_tokenize(i) for i in textFeatures['commentfile5'].fillna('')]

In [ ]:
#Run NER on all tokenized comment file text
print('1')
textFeatures['commentfile1_NER'] = [st.tag(text) for text in textFeatures['commentfile1_tokenized']]
print('2')
textFeatures['commentfile2_NER'] = [st.tag(text) for text in textFeatures['commentfile2_tokenized']]
print('3')
textFeatures['commentfile3_NER'] = [st.tag(text) for text in textFeatures['commentfile3_tokenized']]
print('4')
textFeatures['commentfile4_NER'] = [st.tag(text) for text in textFeatures['commentfile4_tokenized']]
print('5')
textFeatures['commentfile5_NER'] = [st.tag(text) for text in textFeatures['commentfile5_tokenized']]

In [ ]:
textFeatures.to_csv(TEXT_FILE)

In [ ]:
#Specifically, look for length of different NER categories in comment text
textFeatures['commentfile1_NER_PERSON'] = [len([1 for (word,entity) in nerList if entity=="PERSON"]) for nerList in textFeatures['commentfile1_NER']]
textFeatures['commentfile1_NER_ORG'] = [len([1 for (word,entity) in nerList if entity=="ORGANIZATION"]) for nerList in textFeatures['commentfile1_NER']]
textFeatures['commentfile1_NER_LOCATION'] = [len([1 for (word,entity) in nerList if entity=="LOCATION"]) for nerList in textFeatures['commentfile1_NER']]

In [ ]:
textFeatures['commentfile2_NER_PERSON'] = [nerList.count(", 'PERSON')") for nerList in textFeatures['commentfile2_NER']]
textFeatures['commentfile2_NER_ORG'] = [nerList.count(", 'ORGANIZATION')") for nerList in textFeatures['commentfile2_NER']]
textFeatures['commentfile2_NER_LOCATION'] = [nerList.count(", 'LOCATION')") for nerList in textFeatures['commentfile2_NER']]

In [ ]:
textFeatures['commentfile3_NER_PERSON'] = [len([1 for (word,entity) in nerList if entity=="PERSON"]) for nerList in textFeatures['commentfile3_NER']]
textFeatures['commentfile3_NER_ORG'] = [len([1 for (word,entity) in nerList if entity=="ORGANIZATION"]) for nerList in textFeatures['commentfile3_NER']]
textFeatures['commentfile3_NER_LOCATION'] = [len([1 for (word,entity) in nerList if entity=="LOCATION"]) for nerList in textFeatures['commentfile3_NER']]

In [ ]:
textFeatures['commentfile4_NER_PERSON'] = [nerList.count(", 'PERSON')") for nerList in textFeatures['commentfile4_NER']]
textFeatures['commentfile4_NER_ORG'] = [nerList.count(", 'ORGANIZATION')") for nerList in textFeatures['commentfile4_NER']]
textFeatures['commentfile4_NER_LOCATION'] = [nerList.count(", 'LOCATION')") for nerList in textFeatures['commentfile4_NER']]

In [ ]:
textFeatures['commentfile5_NER_PERSON'] = [len([1 for (word,entity) in nerList if entity=="PERSON"]) for nerList in textFeatures['commentfile5_NER']]
textFeatures['commentfile5_NER_ORG'] = [len([1 for (word,entity) in nerList if entity=="ORGANIZATION"]) for nerList in textFeatures['commentfile5_NER']]
textFeatures['commentfile5_NER_LOCATION'] = [len([1 for (word,entity) in nerList if entity=="LOCATION"]) for nerList in textFeatures['commentfile5_NER']]

In [ ]:
#Combine all captions NER results
textFeatures['all_captions_NER_PERSON'] = textFeatures['commentfile1_NER_PERSON'] + \
    textFeatures['commentfile2_NER_PERSON'] + textFeatures['commentfile3_NER_PERSON'] + \
    textFeatures['commentfile4_NER_PERSON'] + textFeatures['commentfile5_NER_PERSON']
textFeatures['all_captions_NER_ORG'] = textFeatures['commentfile1_NER_ORG'] + textFeatures['commentfile2_NER_ORG'] + \
    textFeatures['commentfile3_NER_ORG'] + textFeatures['commentfile4_NER_ORG'] + textFeatures['commentfile5_NER_ORG']
textFeatures['all_captions_NER_LOCATION'] = textFeatures['commentfile1_NER_LOCATION'] + \
    textFeatures['commentfile2_NER_LOCATION'] + textFeatures['commentfile3_NER_LOCATION'] + \
    textFeatures['commentfile4_NER_LOCATION'] + textFeatures['commentfile5_NER_LOCATION']

In [ ]:
all_captions = []
for it,row in textFeatures.iterrows():
    one = row.commentfile1
    if one != one:
        one = ''
    two = row.commentfile2
    if two != two:
        two = ''
    three = row.commentfile3
    if three != three:
        three = ''
    four = row.commentfile4
    if four != four:
        four = ''
    five = row.commentfile5
    if five != five:
        five = ''
    all_captions.append(one + ' ' + two + ' ' + three + ' ' + four + ' ' + five)
textFeatures['all_captions'] = all_captions

In [ ]:
liwcLabels = [i for i in textFeatures['all_captions_liwc'][0][:textFeatures['all_captions_liwc'][0].rfind('\n')+1].split(' ') \
 if i[:4]=="LIWC" or i[:3]=="MRC"]
len(liwcLabels)

In [ ]:
for i in range(len(liwcLabels)):
    textFeatures['all_captions_'+liwcLabels[i]] = \
        [liwcArray[liwcArray.rfind('\n')+1:].split(',')[i] for liwcArray in textFeatures['all_captions_liwc']]

In [ ]:
textFeatures['all_captions_num_words'] = [len([i for i in nltk.word_tokenize(all_captions) if i.isalpha()]) \
                                          for all_captions in textFeatures['all_captions']]

In [ ]:
textFeatures['all_captions_long_words'] = [len([i for i in nltk.word_tokenize(all_captions) if len(i) > 6]) \
                                          for all_captions in textFeatures['all_captions']]

In [ ]:
textFeatures['all_captions_num_numbers'] = [len([i for i in nltk.word_tokenize(all_captions) if i.isdigit()]) \
                                          for all_captions in textFeatures['all_captions']]

In [ ]:
imp.reload(readability)
stats = ['FRE','ARI','FK','CL','GFI','SMOG']
for statI in range(len(stats)):
    for i in range(1,6):
        textFeatures['commentfile' + str(i) + '_' + stats[statI]] = [readability.compute_readability(caption)[statI] \
                                                    for caption in textFeatures['commentfile' + str(i)].fillna('')]
    textFeatures['all_captions_' + stats[statI]] = textFeatures['commentfile1_' + stats[statI]] + \
        textFeatures['commentfile2_' + stats[statI]] + textFeatures['commentfile3_' + stats[statI]] + \
        textFeatures['commentfile4_' + stats[statI]] + textFeatures['commentfile5_' + stats[statI]]

In [ ]:
textFeatures.to_csv(TEXT_FILE)

In [ ]:
textFeatures[['commentfile1','commentfile2','commentfile3','commentfile4','commentfile5']] = \
    textFeatures[['commentfile1','commentfile2','commentfile3','commentfile4','commentfile5']].fillna('')
textFeatures['all_captions'] = textFeatures['commentfile1'] + ' ' + textFeatures['commentfile2'] + ' ' + \
    textFeatures['commentfile3'] + ' ' + textFeatures['commentfile4'] + ' ' + textFeatures['commentfile5']

In [ ]:
#unstemmed unigrams
textFeatures['all_captions_tokenized'] = [nltk.word_tokenize(i) for i in textFeatures['all_captions']]

#stemmed unigrams
lancaster = nltk.LancasterStemmer()
textFeatures['all_captions_stemmed'] = [[lancaster.stem(token) for token in tokenizedList] for tokenizedList in \
                           textFeatures['all_captions_tokenized']]

#stemmed bigrams
ngrams_words = [ngrams(i,2) for i in textFeatures['all_captions_stemmed']]
textFeatures['all_captions_bigrams'] = [[w for w in ngrams_word] for ngrams_word in ngrams_words]

#stemmed trigrams
ngrams_words = [ngrams(i,3) for i in textFeatures['all_captions_stemmed']]
textFeatures['all_captions_trigrams'] = [[w for w in ngrams_word] for ngrams_word in ngrams_words]

In [ ]:
#Calculate POS unigrams, bigrams, and trigrams
tagged_text = [nltk.pos_tag(i) for i in textFeatures['all_captions_tokenized']]
tagger_list = [[w[1] for w in text] for text in tagged_text]
ngrams_words = [ngrams(t_list,2) for t_list in tagger_list]
textFeatures['all_captions_pos_bigrams'] = [[w for w in ngrams_word] for ngrams_word in ngrams_words]

ngrams_words = [ngrams(t_list,1) for t_list in tagger_list]
textFeatures['all_captions_pos_unigrams'] = [[w for w in ngrams_word] for ngrams_word in ngrams_words]

ngrams_words = [ngrams(t_list,3) for t_list in tagger_list]
textFeatures['all_captions_pos_trigrams'] = [[w for w in ngrams_word] for ngrams_word in ngrams_words]

In [ ]:
textFeatures.to_csv(TEXT_FILE)

In [ ]:
#Returns a dataframe where the rows are the students and the columns are the vocabulary elements
#An __oov vocabulary element is included at the end, for all out of vocabulary words
#
#Each element in the dataframe is the count of how many times that vocabulary element appears in the student's 
#writing
def calculateVocabulary(column,minWordCount=5):
    word_set = Counter()
    for vocab_list in np.ravel(textFeatures[column]):
        for word in vocab_list:
            word_set[word] += 1

    vocab = []
    oov = []
    for word,count in word_set.items():
        if count >= minWordCount:
            vocab.append(word)
        else:
            oov.append(word)

    print("There are %d unique vocabulary elements and %d out of vocabulary elements for the %s column" \
          % (len(vocab),len(oov),column))

    df = pd.DataFrame(index = textFeatures.index, columns = vocab+['__oov'])
    df = df.fillna(value=0)

    for word in vocab:
        df[word] = [i.count(word) for i in textFeatures[column]]
    for word in oov:
        df['__oov'] += [i.count(word) for i in textFeatures[column]]
        
    return df

In [ ]:
unigrams = calculateVocabulary('all_captions_stemmed')
unigrams['userid'] = textFeatures['id']
unigrams.to_csv(home+"text_features/all_captions_unigrams.csv")

bigrams = calculateVocabulary('all_captions_bigrams')
bigrams['userid'] = textFeatures['id']
bigrams.to_csv(home+"text_features/all_captions_bigrams.csv")

trigrams = calculateVocabulary('all_captions_trigrams')
trigrams['userid'] = textFeatures['id']
trigrams.to_csv(home+"text_features/all_captions_trigrams.csv")

pos_unigrams = calculateVocabulary('all_captions_pos_unigrams')
pos_unigrams['userid'] = textFeatures['id']
pos_unigrams.to_csv(home+"text_features/all_captions_pos_unigrams.csv")

pos_bigrams = calculateVocabulary('all_captions_pos_bigrams')
pos_bigrams['userid'] = textFeatures['id']
pos_bigrams.to_csv(home+"text_features/all_captions_pos_bigrams.csv")

pos_trigrams = calculateVocabulary('all_captions_pos_trigrams')
pos_trigrams['userid'] = textFeatures['id']
pos_trigrams.to_csv(home+"text_features/all_captions_pos_trigrams.csv")

In [ ]:
#char_unigrams
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(1, 1), min_df=5)
counts = ngram_vectorizer.fit_transform(textFeatures['all_captions'])

char_unigrams = pd.DataFrame(data=counts.toarray().astype(int),index=textFeatures.index,
                             columns=ngram_vectorizer.get_feature_names())
char_unigrams['id'] = textFeatures['id']
char_unigrams.to_csv(home+"text_features/all_captions_char_unigrams.csv")

#char_bigrams
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2), min_df=5)
counts = ngram_vectorizer.fit_transform(textFeatures['all_captions'])

char_bigrams = pd.DataFrame(data=counts.toarray().astype(int),index=textFeatures.index,
                             columns=ngram_vectorizer.get_feature_names())
char_bigrams['id'] = textFeatures['id']
char_bigrams.to_csv(home+"text_features/all_captions_char_bigrams.csv")

#char_trigrams
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(3, 3), min_df=5)
counts = ngram_vectorizer.fit_transform(textFeatures['all_captions'])

char_trigrams = pd.DataFrame(data=counts.toarray().astype(int),index=textFeatures.index,
                             columns=ngram_vectorizer.get_feature_names())
char_trigrams['id'] = textFeatures['id']
char_trigrams.to_csv(home+"text_features/all_captions_char_trigrams.csv")